In [1]:
%use fuel(2.3.1)

In [2]:
val envs = java.io.File("../../.env")
    .readLines()
    .map {
        it.split("=")[0] to it.split("=")[1].trim('"')
    }.toMap()

In [3]:
val session = envs.get("AOC_SESSION")
val year = 2021
val day = 21

In [4]:
fun getInput(year: Int, day: Int, session: String): String {
    val (_, _, result) = "https://adventofcode.com/$year/day/$day/input"
    .httpGet()
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get().trim()
}

In [5]:
fun submitAnswer(year: Int, day: Int, session: String, level: Int, answer: String): String {
    val (_, _, result) = Fuel
    .post(
        "https://adventofcode.com/$year/day/$day/answer", 
        parameters = listOf("level" to level, "answer" to answer))
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [6]:
val sample = """Player 1 starting position: 4
Player 2 starting position: 8"""

In [7]:
val input = getInput(year, day, session)

In [8]:
fun partOne(input: String): String {
    class Die {
        var rolls = 0
        var v = 1
        
        fun roll(times: Int): Int {
            var sum = 0
            repeat(times) {
                sum += v
                v++
                rolls++
                if (v > 100) v = 1
            }
            return sum
        }
    }
    
    var (a, b) = input.split("\n").map {
        it.split(": ")[1].toInt()
    }
    var pa = 0
    var pb = 0
    val die = Die()
    
    while (pa < 1000 && pb < 1000) {
        a = (a + die.roll(3) - 1) % 10 + 1
        pa += a
        if (pa >= 1000) break
        
        b = (b + die.roll(3) - 1) % 10 + 1
        pb += b
    }
    
    return (pa.coerceAtMost(pb) * die.rolls).toString()
}

In [9]:
partOne(sample)

739785

In [10]:
val partOneAns = partOne(input)
partOneAns

1006866

In [ ]:
submitAnswer(year, day, session, 1, partOneAns)

In [11]:
fun partTwo(input: String): String {
    val (a, b) = input.split("\n").map {
        it.split(": ")[1].toInt()
    }
    
    val dp = Array(22) { Array(22) { Array(11) { Array(11) { LongArray(2) } } } }
    dp[0][0][a][b][0] = 1
    val coeff = listOf<Long>(0, 0, 0, 1, 3, 6, 7, 6, 3, 1)
    
    for (pa in 0 until 21) for (pb in 0 until 21) for (q in 0..1) for (a in 1..10) for (b in 1..10) {
        if (dp[pa][pb][a][b][q] == 0L) continue
        
        if (q == 0) {
            for (die in 3..9) {
                val a1 = (a + die - 1) % 10 + 1
                val pa1 = (pa + a1).coerceAtMost(21)
                dp[pa1][pb][a1][b][1 - q] += dp[pa][pb][a][b][q] * coeff[die]
            }
        } else {
            for (die in 3..9) {
                val b1 = (b + die - 1) % 10 + 1
                val pb1 = (pb + b1).coerceAtMost(21)
                dp[pa][pb1][a][b1][1 - q] += dp[pa][pb][a][b][q] * coeff[die]
            }
        }
    }
    
    var aWin = 0L
    var bWin = 0L
    for (pb in 0 until 21) for (q in 0..1) for (a in 1..10) for (b in 1..10) aWin += dp[21][pb][a][b][q]
    for (pa in 0 until 21) for (q in 0..1) for (a in 1..10) for (b in 1..10) bWin += dp[pa][21][a][b][q]
    
    return (aWin.coerceAtLeast(bWin)).toString()
}

In [12]:
partTwo(sample)

444356092776315

In [13]:
val partTwoAns = partTwo(input)
partTwoAns

273042027784929

In [ ]:
submitAnswer(year, day, session, 2, partTwoAns)